<a href="https://colab.research.google.com/github/Raquib603/Basic_RoBerta_next/blob/main/SamLowe_Roberta_next.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
# model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"SamLowe/roberta-base-go_emotions"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)



#Taking Input

In [ ]:
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) neutral 0.8032
2) approval 0.1216
3) excitement 0.0297
4) admiration 0.0145
5) realization 0.0093
6) joy 0.0038
7) optimism 0.0026
8) annoyance 0.0024
9) surprise 0.0018
10) desire 0.0015
11) pride 0.001
12) love 0.0009
13) amusement 0.0008
14) disappointment 0.0007
15) relief 0.0007
16) fear 0.0007
17) disapproval 0.0007
18) caring 0.0006
19) curiosity 0.0006
20) disgust 0.0006
21) anger 0.0005
22) sadness 0.0004
23) confusion 0.0004
24) nervousness 0.0003
25) gratitude 0.0003
26) embarrassment 0.0002
27) grief 0.0001
28) remorse 0.0001


#Making Function for the model input and that return a output

In [ ]:
def final_sentiment_scores(comments_reviews):
  text = preprocess(comments_reviews)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  # # TF
  # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
  # model.save_pretrained(MODEL)
  # text = "Covid cases are increasing fast!"
  # encoded_input = tokenizer(text, return_tensors='tf')
  # output = model(encoded_input)
  # scores = output[0][0].numpy()
  # scores = softmax(scores)
  # Print labels and scores
  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  #only for 4 top values
  #for i in range(scores.shape[0]):

  for i in range(3):
      l = config.id2label[ranking[i]]
      s = scores[ranking[i]]
      print(f"{i+1}) {l} {np.round(float(s), 4)}",  end = ' ')



In [ ]:
#Checking examples with my created function
Comment = "you are a good boy"
final_sentiment_scores(Comment)

1) admiration 0.9925 2) approval 0.0029 3) neutral 0.0013 4) joy 0.0003 

#CUSTOMIZED Output

In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
def clean_token(token):
    if token is not None:  # Check if token is not None
        return token.replace('Ġ', '')  # Replace Ġ with nothing (removes the space indicator)
    return ''  # Return an empty string if token is None

In [ ]:
RED = "\033[91m"
RESET = "\033[0m"

In [ ]:
def final_sentiment_scores(comments_reviews):
    text = preprocess(comments_reviews)
    encoded_input = tokenizer(text, return_tensors='pt')

    # Forward pass through the model
    output = model(**encoded_input)
    logits = output[0][0].detach().numpy()
    scores = softmax(logits)

    # Get the top 4 emotions
    ranking = np.argsort(scores)[::-1]

    # Print top 4 emotions and their scores
    print("Top 4 emotions with scores:")
    for i in range(4):
        label = config.id2label[ranking[i]]
        score = scores[ranking[i]]
        print(f"{i+1}) {label}: {np.round(float(score), 4)}")

    # Now identify the tokens contributing to emotions
    tokens = tokenizer.convert_ids_to_tokens(encoded_input["input_ids"][0])

    # Token importance heuristic based on logits contribution
    top_emotion_index = ranking[0]  # Focus on the top emotion first
    token_contributions = logits[:len(tokens)]  # Simplified contribution (logits)

    # Rank tokens by their contribution scores
    ranked_tokens = np.argsort(token_contributions)[::-1]  # Sort tokens by importance

    # Highlight only the top emotionally contributing tokens
    top_token_indices = ranked_tokens[:4]  # Select top 4 contributing tokens

    # Create a highlighted version of the sentence with red color


    highlighted_sentence = ' '.join(
        [f"{RED}{tokens[i]}{RESET}" if i in top_token_indices else tokens[i] for i in range(len(tokens))]

    )



    print("\nHighlighted sentence with emotional text:")
    #call the clean function to clean my output
    print(clean_token(highlighted_sentence))

    return highlighted_sentence, scores, ranking

In [ ]:
input_text = "One person wrote five paragraphs detailing their concerns about homeless people at Stranahan Park, saying that a man had recently chased after their toy poodle."

#store output in a different variable and then pass it through clean_token
x = final_sentiment_scores(input_text)



Top 4 emotions with scores:
1) neutral: 0.9869
2) approval: 0.0023
3) annoyance: 0.002
4) caring: 0.0018

Highlighted sentence with emotional text:
<s> One person wrote five paragraphs detailing their concerns about homeless people at St ran ahan Park , saying that a man had recently chased after their toy p oodle . </s>
